In [109]:
from itertools import permutations
import random
import math
from numpy.random import randint
from numpy.random import rand

# ALGORITMO GENETICO

### GENERAZIONE POPOLAZIONE INIZIALE

In [117]:
A = [('a1',9), ('a2',6), ('a3',3), ('a4',8), ('a5',5)]

B = [('b1',5), ('b2',7), ('b3',2), ('b4',10)]

jobs = A + B


In [118]:
def generate_population(jobs, n_pop):
    permutazioni = list(permutations(jobs))
    # Seleziona casualmente n_pop permutazioni
    population = random.sample(permutazioni, n_pop)
    return population

### FITNESS FUNCTION

In [119]:
# fitness function equivale alla nostra funzione obbiettivo (che deve essere minimizzata)
def fitness_function(soluzione, alfa=0.3):
    somma_A = 0
    somma_B = 0
    somma_tot = 0
    for job in soluzione:
        string = str(job[0])
        somma_tot += job[1]
        if string.startswith('a'):
            somma_A += somma_tot
        else:
            somma_B += somma_tot
    
    return (alfa * (somma_A + somma_B)) + ((1-alfa) * abs((somma_A - somma_B)))

### SELEZIONE

In [120]:
# tournament selection
def selection(population, scores, k=3):
    # primo scelto randomicamente (utile estrarlo prima per fare i confronti)
    selection_ix = randint(len(population))
    # scegliamo gli altri k-1 partecipanti randomicamente
    for ix in randint(0, len(population), k-1):
        # controlliamo chi è il migliore tra i k partecipanti
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return population[selection_ix]

### CROSSOVER

In [131]:
def crossover(p1, p2, r_cross):
    # i figli sono copie dei genitori di default
    c1, c2 = tuple(p1), tuple(p2)

    if rand() < r_cross:
        # selezioniamo il punto di taglio per il crossover
        pt = randint(1, len(p1)-1)
        # crossover
        c1 = p1[:pt] + tuple([elem for elem in p2 if elem not in p1[:pt]])
        c2 = p2[:pt] + tuple([elem for elem in p1 if elem not in p2[:pt]])
    return c1, c2

### MUTAZIONE

In [134]:
# mutazione: scambio di posizione di due job
def mutation(c, r_mut):
        c1= c[0]
        c2 = c[1]
        if rand() < r_mut:
             # Selezione casuale di due indici di due job in c1
             indexes = random.sample(range(len(c1)), 2)
             i, j = indexes[0], indexes[1]
             # Scambia gli elementi di posizione
             c1_list = list(c1)
             c1_list[i], c1_list[j] = c1_list[j], c1_list[i]
             c1 = tuple(c1_list)

             # Selezione casuale di due indici di due job in c2
             indexes = random.sample(range(len(c2)), 2)
             i, j = indexes[0], indexes[1]
             # Scambia gli elementi di posizione
             c2_list = list(c2)
             c2_list[i], c2_list[j] = c2_list[j], c2_list[i]
             c2 = tuple(c2_list)




### ALGORITMO

In [135]:
def genetic_algorithm(jobs, n_iter, n_pop, r_cross, r_mut):
    # popolazione iniziale
    population = generate_population(jobs, n_pop)
    # teniamo traccia della migliore soluzione (all'inizio prendiamo la prima soluzione)
    best, best_eval = 0, fitness_function(population[0])
    # enumeriamo le iterazioni (il numero di generazioni create)
    for gen in range(n_iter):
        # valutiamo tutti i candidati della popolazione
        scores = [fitness_function(solution) for solution in population]
        # vediamo qual'è la migliore soluzione della generazione corrente
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = population[i], scores[i]
                print(">%d, new best f(%s) = %.3f" % (gen,  population[i], scores[i]))
        # applichiamo la selezione: selezioniamo i genitori della nuova generazione (tanti genitori quanto la popolazione è grande)
        selected = [selection(population, scores) for _ in range(n_pop)]
        # creiamo la nuova generazione
        children = list()
        # per ogni coppia di genitori
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i+1]
            # crossover
            #c è una coppia di figli: 2 figli ogni 2 genitori --> la dimensione della popolazione rimane la stessa
            for c in crossover(p1, p2, r_cross):
                # mutazione
                mutation(c, r_mut)
                # memorizziamo la nuova generazione
                children.append(c)
        # sostituiamo la nuova generazione
        population = children
    return [best, best_eval]

In [137]:
# numero totale di iterazioni
n_iter = 100
# dimensione popolazione
n_pop = 100
# soglia crossover
r_cross = 0.9
# soglia mutazione
r_mut = 0.5 
# gli altri parametri da stabilire sono alfa e k

best,best_eval = genetic_algorithm(jobs, n_iter, n_pop, r_cross, r_mut)

>0, new best f((('b3', 2), ('a2', 6), ('b1', 5), ('a3', 3), ('b2', 7), ('b4', 10), ('a5', 5), ('a4', 8), ('a1', 9))) = 134.600
>0, new best f((('a2', 6), ('b1', 5), ('a1', 9), ('a5', 5), ('b3', 2), ('b4', 10), ('b2', 7), ('a3', 3), ('a4', 8))) = 105.400
>0, new best f((('b3', 2), ('a1', 9), ('a2', 6), ('b1', 5), ('a5', 5), ('a4', 8), ('b2', 7), ('a3', 3), ('b4', 10))) = 86.600
>0, new best f((('b3', 2), ('a4', 8), ('a3', 3), ('a5', 5), ('b1', 5), ('a1', 9), ('a2', 6), ('b4', 10), ('b2', 7))) = 83.600
>0, new best f((('a3', 3), ('b3', 2), ('b4', 10), ('a5', 5), ('a2', 6), ('a4', 8), ('a1', 9), ('b1', 5), ('b2', 7))) = 76.800
>1, new best f((('b1', 5), ('a3', 3), ('a1', 9), ('a2', 6), ('b3', 2), ('a5', 5), ('b4', 10), ('a4', 8), ('b2', 7))) = 76.000
>1, new best f((('b1', 5), ('a3', 3), ('a5', 5), ('a2', 6), ('b3', 2), ('a1', 9), ('b2', 7), ('a4', 8), ('b4', 10))) = 72.000
>3, new best f((('b3', 2), ('a4', 8), ('a3', 3), ('a5', 5), ('b1', 5), ('a2', 6), ('b2', 7), ('a1', 9), ('b4', 10)))

# TEST DELLE FUNZIONI

In [108]:
A = [('a1',9), ('a2',6), ('a3',3), ('a4',8), ('a5',5)]

B = [('b1',5), ('b2',7), ('b3',2), ('b4',10)]

jobs = A + B

jobs



[('a1', 9),
 ('a2', 6),
 ('a3', 3),
 ('a4', 8),
 ('a5', 5),
 ('b1', 5),
 ('b2', 7),
 ('b3', 2),
 ('b4', 10)]

### TEST GENERAZIONE POPOLAZIONE

In [22]:
soluzioni = generate_population(jobs, n_pop)
soluzioni

[(('b1', 5),
  ('a2', 6),
  ('a5', 5),
  ('a4', 8),
  ('b2', 7),
  ('b4', 10),
  ('a3', 3),
  ('a1', 9),
  ('b3', 2)),
 (('a5', 5),
  ('a1', 9),
  ('b3', 2),
  ('a3', 3),
  ('a4', 8),
  ('a2', 6),
  ('b2', 7),
  ('b4', 10),
  ('b1', 5)),
 (('a5', 5),
  ('a4', 8),
  ('b4', 10),
  ('a2', 6),
  ('a1', 9),
  ('b3', 2),
  ('b2', 7),
  ('b1', 5),
  ('a3', 3)),
 (('b2', 7),
  ('a1', 9),
  ('b1', 5),
  ('a5', 5),
  ('b3', 2),
  ('a4', 8),
  ('a3', 3),
  ('b4', 10),
  ('a2', 6)),
 (('b4', 10),
  ('b1', 5),
  ('a3', 3),
  ('a2', 6),
  ('a5', 5),
  ('a4', 8),
  ('b2', 7),
  ('a1', 9),
  ('b3', 2)),
 (('b3', 2),
  ('b4', 10),
  ('a3', 3),
  ('b1', 5),
  ('a4', 8),
  ('a5', 5),
  ('a1', 9),
  ('b2', 7),
  ('a2', 6)),
 (('b4', 10),
  ('a5', 5),
  ('a3', 3),
  ('a2', 6),
  ('b3', 2),
  ('a4', 8),
  ('b2', 7),
  ('b1', 5),
  ('a1', 9)),
 (('a3', 3),
  ('a2', 6),
  ('b2', 7),
  ('a4', 8),
  ('a1', 9),
  ('a5', 5),
  ('b1', 5),
  ('b3', 2),
  ('b4', 10)),
 (('a4', 8),
  ('b4', 10),
  ('b2', 7),
  ('a5',

In [23]:
len(soluzioni)

100

### TEST FITNESS FUNCTION

In [35]:
jobs

[('a1', 9),
 ('a2', 6),
 ('a3', 3),
 ('a4', 8),
 ('a5', 5),
 ('b1', 5),
 ('b2', 7),
 ('b3', 2),
 ('b4', 10)]

In [27]:
# alfa = 0.5, mi aspetto 179
val = fitness_function(jobs)
val

179.0

In [29]:
#alfa 0.3, mi aspetto 139.4
val = fitness_function(jobs)
val

139.39999999999998

### TEST CROSSOVER

In [101]:
sol1 = [('b2', 7), ('a3', 3), ('b1', 5), ('a1', 9), ('b4', 10), ('b3', 2), ('a4', 8), ('a2', 6), ('a5', 5)]
sol2 = [('b1', 5), ('b4', 10), ('a2', 6), ('b2', 7), ('a3', 3), ('b3', 2), ('a1', 9), ('a4', 8), ('a5', 5)]


In [102]:
c = crossover(sol1,sol2,1)

In [103]:
c[0]

[('b2', 7),
 ('a3', 3),
 ('b1', 5),
 ('a1', 9),
 ('b4', 10),
 ('a2', 6),
 ('b3', 2),
 ('a4', 8),
 ('a5', 5)]

In [104]:
c[1]

[('b1', 5),
 ('b4', 10),
 ('a2', 6),
 ('b2', 7),
 ('a3', 3),
 ('a1', 9),
 ('b3', 2),
 ('a4', 8),
 ('a5', 5)]

### TEST MUTAZIONE

In [105]:
mutation(c,1)

In [106]:
# prima era : [('b2', 7),('a3', 3),('b1', 5),('a1', 9),('b4', 10),('a2', 6),('b3', 2),('a4', 8),('a5', 5)]
c[0]

[('b2', 7),
 ('a3', 3),
 ('b1', 5),
 ('b4', 10),
 ('a1', 9),
 ('a2', 6),
 ('b3', 2),
 ('a4', 8),
 ('a5', 5)]

In [107]:
#prima era: [('b1', 5),('b4', 10),('a2', 6),('b2', 7),('a3', 3),('a1', 9),('b3', 2),('a4', 8),('a5', 5)]
c[1]

[('b1', 5),
 ('b4', 10),
 ('a2', 6),
 ('b3', 2),
 ('a3', 3),
 ('a1', 9),
 ('b2', 7),
 ('a4', 8),
 ('a5', 5)]